In [ ]:
# Import required libraries
import numpy as np # numpy is used for handling numerical operations on arrays
import pandas as pd # pandas is used for handling and manipulating structured data
from torchvision.io import read_image # For reading image data from a file
from torch.utils.data import Dataset, DataLoader # For creating custom datasets and dataloaders
from torchvision.transforms import ToTensor 
import torch # PyTorch library for neural network operations
import torchvision # a package consists of popular datasets, model architectures, and common image transformations for computer vision.
from torchvision import transforms # for performing transformations on our dataset
import matplotlib.pyplot as plt # for plotting graphs and visualizing data
from torch.utils.data import random_split # for splitting the dataset into train and validation
import os # useful for operating system operations

# Define the custom dataset
class MNISTxCIFAR(Dataset):
    def __init__(self, img_dir, transform=None, train=True):
        self.img_dir = img_dir # The directory where our images are stored
        self.transform = transform # The transformations to be applied on the images
        self.labels = [] # List to hold the labels of the images

        if train: # If this is the training set
            self.classes = os.listdir(img_dir) # Get the list of classes (folders) in the directory
            self.classes.sort() # Sort the classes for consistency
            self.img_paths = [] # List to hold the paths of the images
            self.labels = [] # Reset the labels list
            for c in self.classes: # For each class
                c_dir = os.path.join(img_dir, c) # Get the directory of the class
                c_imgs = os.listdir(c_dir) # Get the list of images in the class directory
                for img in c_imgs: # For each image
                    self.img_paths.append(os.path.join(c_dir, img)) # Add the path of the image to the list
                    self.labels.append(int(c)) # Add the class label to the labels list
        else: # If this is the test set
            self.img_paths = [os.path.join(img_dir, img) for img in os.listdir(img_dir)] 
            self.img_paths.sort()
            # No labels are included for the test set

    def __len__(self):
        return len(self.img_paths) # The length of the dataset is the number of images

    def __getitem__(self, idx): # Method to get an item from the dataset
        img_path = self.img_paths[idx] # Get the path of the image
        image = read_image(img_path) # Read the image from the path
        if self.transform: # If there are any transformations to be applied
            image = self.transform(image) # Apply the transformations
        if self.labels: # If there are labels
            return image, self.labels[idx] # Return the image and its label
        else:
            return image  # For the test set, we return just the image

# Training and testing transformations
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(), #TODO: Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(), #TODO: Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dir = "/kaggle/input/mnistxcifar-competition-oxford/MNISTxCIFAR/train"
test_dir = "/kaggle/input/mnistxcifar-competition-oxford/MNISTxCIFAR/test"

# Load the training data
full_train_dataset = MNISTxCIFAR(train_dir, transform=train_transform) # Create the full training dataset

# Split the full training dataset into training and validation sets
# Let's say we want to use 80% of the samples for training and 20% for validation

# TODO: Split the dataset based on 80-20 split
train_size = int(0.8 * len(full_train_dataset))  # 80% of the dataset size
val_size = len(full_train_dataset) - train_size  # The remaining samples

train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Batch_size
batch_size = 32

# Create data loaders 
# TODO: Fill in the loader parameters
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # Create the training dataloader
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) # Create the validation dataloader

# Load the test data
# TODO: Fill in the loader parameters
test_dataset = MNISTxCIFAR(test_dir, train=False, transform=test_transform) # Create the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # Create the test dataloader

# Check out the datasets
for images, labels in train_dataloader: # For each batch in the training dataloader
    print(images.shape, labels.shape) # Print the shapes of the images and labels tensors
    break # Stop after the first batch

for images, labels in val_dataloader: # For each batch in the validation dataloader
    print(images.shape, labels.shape) # Print the shapes of the images and labels tensors
    break # Stop after the first batch

for images in test_dataloader: # For each batch in the test dataloader
    print(images.shape) # Print the shape of the images tensor
    break # Stop after the first batch

In [ ]:
def imshow(inp, mean, std, title=None):
    """Imshow for Tensor after reversing normalization."""
    inp = inp.numpy().transpose((1, 2, 0))
    # Reverse the normalization
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])


# Create a subplot grid of 10 rows and 6 columns (total 60 subplots)
fig, axs = plt.subplots(10, 6, figsize=(10, 20))  # 10 classes, 6 images each

# Flatten the array of Axes instances for easy iteration
axs = axs.ravel()

# Dictionary to hold images of each class
class_images = {}

# Iterate over batches of images and labels in the training dataloader
for images, labels in train_dataloader:
    # Iterate over individual images and labels in the batch
    for img, label in zip(images, labels):
        label = label.item()  # Convert the label tensor to a Python scalar
        if label not in class_images:  # If this class hasn't been seen before
            class_images[label] = []  # Create a new list for this class
        if len(class_images[label]) < 6:  # If fewer than 6 images have been collected for this class
            class_images[label].append(img)  # Add the current image to the class's list
        if all(len(imgs) >= 6 for imgs in class_images.values()):  # If 6 images have been collected for all classes
            break  # Break out of the loop over images and labels
    else:  # If the loop over images and labels wasn't broken
        continue  # Continue with the next batch
    break  # If the loop over images and labels was broken, break the loop over batches


# Iterate over the collected images of each class
for label, imgs in class_images.items():
    for i, img in enumerate(imgs):  # For each image
        img = img.numpy().transpose((1, 2, 0))  # Convert to numpy array and transpose
        img = std * img + mean  # Reverse normalization
        img = np.clip(img, 0, 1)  # Clip values to range [0, 1]
        axs[label*6 + i].imshow(img)  # Display the image in the appropriate subplot
        axs[label*6 + i].axis('off')  # Turn off the axis
        if i == 0:  # For the first image of each class
            axs[label*6 + i].set_title(f'Class: {label}')  # Set the subplot's title to the class label
plt.show()  # Display the plot

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.optim import lr_scheduler
import torch.nn.functional as F
from tqdm import tqdm
from torchsummary import summary

class MyNetwork(nn.Module):
    # Create your neural network!
    def __init__(self, input_shape, in_channels, num_classes):
        super(MyNetwork, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 16, kernel_size = 5, padding = 2, stride = 1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(16, 32, 5, padding = 2, stride = 1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(32, 64, 5, padding = 2, stride = 1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(64 * (input_shape//8)*(input_shape//8), 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.fc_out = nn.Linear(16, num_classes)

        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.pool1(self.activation(self.bn1(self.conv1(x))))
        x = self.pool2(self.activation(self.bn2(self.conv2(x))))
        x = self.pool3(self.activation(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)

        x = F.dropout(self.activation(self.fc1(x)), p=0.5, training=self.training)
        x = F.dropout(self.activation(self.fc2(x)), p=0.5, training=self.training)
        x = F.dropout(self.activation(self.fc3(x)), p=0.5, training=self.training)
        x = F.dropout(self.activation(self.fc4(x)), p=0.5, training=self.training)
        x = self.activation(self.fc5(x))
        output = self.fc_out(x)
        return output

# model instance
model = MyNetwork(input_shape = input_shape, in_channels = in_channels, num_classes = num_classes)

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# Loss function and optimizer
# TODO: Select a criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.1)

# Training loop
# TODO: Select your training epochs

num_epochs = 16
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model.train()

    for inputs, labels in tqdm(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass through the model
        outputs = model(inputs)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward + optimize
        loss.backward()
        optimizer.step()
    
    # Update the learning rate
    scheduler.step()

    # Check performance on validation set
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(val_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass through the model
            outputs = model(inputs)
            
            # Get the model predictions
            _, predicted = torch.max(outputs.data, 1)
            
            # Compute the accuracy
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss = val_loss / total
    accuracy = 100 * correct / total
    
    # Print the accuracy
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {accuracy:.2f}%")

print('Training complete')

In [ ]:
'''
Do not change this cell! This does the test predictions for you needed for evaluation.
'''

import pandas as pd

# Placeholder for your predictions and image identifiers
predictions = []
image_ids = []

model.eval()
# Iterate over test set
for image in test_dataloader:
    image = image.cuda()
    
    with torch.no_grad():
        # Predict the label using your model (modify this as needed)
        pred = model(image)
    
    # Convert scores to class predictions
    pred = pred.argmax(dim=1)

    # Append to our lists
    predictions.extend(pred.tolist())


In [ ]:
'''
Do not change this cell! This appends your predictions to a submission csv file to get your score.
'''

indices = list(range(len(predictions)))
indices = ['{:07d}'.format(i) for i in indices]

# Create DataFrame
df = pd.DataFrame({
    'ID': indices,  # convert tensor to list
    'Label': predictions
})

# Save DataFrame as CSV without headers
df.to_csv('submission.csv', index=False)

In [ ]:
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool5 = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(512 * 1 * 1, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc_out = nn.Linear(32, 10)

        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.pool1(self.activation(self.bn1(self.conv1(x))))
        x = self.pool2(self.activation(self.bn2(self.conv2(x))))
        x = self.pool3(self.activation(self.bn3(self.conv3(x))))
        x = self.pool4(self.activation(self.bn4(self.conv4(x))))
        x = self.pool5(self.activation(self.bn5(self.conv5(x))))

        x = x.view(x.size(0), -1)

        x = F.dropout(self.activation(self.fc1(x)), p=0.5, training=self.training)
        x = F.dropout(self.activation(self.fc2(x)), p=0.5, training=self.training)
        x = F.dropout(self.activation(self.fc3(x)), p=0.5, training=self.training)
        x = self.activation(self.fc4(x))
        output = self.fc_out(x)
        return output

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.optim import lr_scheduler
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim.lr_scheduler import ExponentialLR

class MyNetwork(nn.Module):
    # Create your neural network!
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5, padding = 2)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(6, 16, 5, padding = 2)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(16, 32, 5, padding = 2)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv4 = nn.Conv2d(32, 64, 5, padding = 2)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(64, 128, 5, padding = 2)
        self.pool5 = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128 * 1 * 1, 120)
        self.fc2 = nn.Linear(120, 90)
        self.fc3 = nn.Linear(90, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc_out = nn.Linear(32, 10)

    def forward(self, x):
        bs = x.shape[0]
        
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = self.pool5(F.relu(self.conv5(x)))

        x = x.view(bs, -1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        output = self.fc_out(x)
        return output

# model instance
model = MyNetwork()

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
# TODO: Select a criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001, amsgrad = False)

# Learning rate scheduler
scheduler = ExponentialLR(optimizer = optimizer, gamma = 0.1, verbose = True)

# Training loop
# TODO: Select your training epochs
num_epochs = 23
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            dataloader = train_dataloader
        else:
            model.eval()
            dataloader = val_dataloader

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(train_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
        
            # Zero the parameter gradients
            optimizer.zero_grad()
        
            # Forward pass through the model
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
        
                # Compute the loss
                loss = criterion(outputs, labels)
                
                # Backward + optimize
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            
            # Check performance
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        # Learning rate scheduler
        if phase == 'train':
            if (epoch + 1) % 7 == 0:
                scheduler.step()

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}%')

print('Training complete')